In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import optuna

/opt/anaconda3/envs/tp_2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ratings = pd.read_csv('../data/ml-32m/ratings.csv')
df_users = pd.read_csv('../data/ml-32m/users.csv')
df_movies = pd.read_csv('../data/ml-32m/movies.csv')

#df_movies.loc[df_movies['IMDB URL'].isna(), 'IMDB URL'] = ''

u_unique = ratings.userId.unique()
user2Idx = {o:i+1 for i,o in enumerate(u_unique)}

m_unique = ratings.movieId.unique()
movie2Idx = {o:i+1 for i,o in enumerate(m_unique)}

ratings.userId = ratings.userId.apply(lambda x: user2Idx[x])

ratings.movieId = ratings.movieId.apply(lambda x: movie2Idx[x])

ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,944249077
1,1,2,1.0,944250228
2,1,3,2.0,943230976
3,1,4,5.0,944249077
4,1,5,5.0,943228858


In [4]:
from sklearn.model_selection import train_test_split
ratings_train, ratings_val = train_test_split(ratings, test_size=0.2)
n_users = int(ratings.userId.nunique())
n_movies = int(ratings.movieId.nunique())
n_users_train = int(ratings_train.userId.nunique())
n_movies_train = int(ratings_train.movieId.nunique())
print(n_users, n_movies, n_users_train, n_movies_train)
max_rating = ratings_train['rating'].max()
min_rating = ratings_train['rating'].min()
av_rating = ratings_train['rating'].mean()
max_rating, min_rating, av_rating

200948 84432 200948 80191


(5.0, 0.5, 3.5402706224956457)

In [5]:
from keras.layers import Input, Embedding, Flatten, Dropout, Concatenate, Dense, Activation, Lambda
from keras import Model
from keras.regularizers import l2
from keras.optimizers import Adam

2024-11-25 23:06:53.320541: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
n_latent_factors_user = 5
#mlflow.log_param("n_latent_factors_user", n_latent_factors_user)
n_latent_factors_movie = 5
#mlflow.log_param("n_latent_factors_movie", n_latent_factors_movie)

In [7]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
checkpointer = ModelCheckpoint(filepath='weights2.hdf5', verbose=1, save_best_only=True, monitor='val_root_mean_squared_error')

In [7]:
patience = 5
early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
#mlflow.log_param("early_stopping_patience", patience)

NameError: name 'EarlyStopping' is not defined

In [8]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',      # Métrica a monitorear (puede ser 'val_loss' o 'loss')
    factor=0.5,              # Factor de reducción del learning rate (e.g., reduce a la mitad)
    patience=5,              # Número de épocas sin mejora antes de reducir
    min_lr=1e-6,             # Learning rate mínimo permitido
    verbose=1                # Mostrar logs cuando se reduzca el LR
)
#mlflow.log_param("reduce_lr", reduce_lr)

In [8]:
import keras.backend as K 
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [8]:
def objective(trial):
    from sklearn.model_selection import train_test_split
    from keras.layers import Input, Embedding, Flatten, Dropout, Concatenate, Dense, Activation, Lambda
    from keras import Model
    from keras.regularizers import l2
    from keras.optimizers import Adam
    
    
    hidden_units = trial.suggest_int("hidden_units", 5, 200)
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 1e1, log=True)
    activation = 'relu'
    optimizer = 'adam'
    dropout_1 = trial.suggest_float("dropout_1", 0.1, 0.4, log=True)
    dropout_2 = trial.suggest_float("dropout_2", 0.1, 0.5, log=True)

    ratings_train, ratings_val = train_test_split(ratings, test_size=0.2)
    n_users = int(ratings.userId.nunique())
    n_movies = int(ratings.movieId.nunique())
    n_users_train = int(ratings_train.userId.nunique())
    n_movies_train = int(ratings_train.movieId.nunique())
    print(n_users, n_movies, n_users_train, n_movies_train)
    max_rating = ratings_train['rating'].max()
    min_rating = ratings_train['rating'].min()
    av_rating = ratings_train['rating'].mean()
    max_rating, min_rating, av_rating


    movie_embedding_regularizer = 0.001
    #mlflow.log_param("movie_embedding_regularizer_l2", movie_embedding_regularizer)

    movie_input = Input(shape=[1],name='Item')
    movie_embedding = Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding', embeddings_regularizer = l2(movie_embedding_regularizer))(movie_input)
    movie_vec = Flatten(name='FlattenMovies')(movie_embedding)
    movie_vec = Dropout(dropout_1)(movie_vec)

    user_input = Input(shape=[1],name='User')
    user_vec = Flatten(name='FlattenUsers')(Embedding(n_users + 1, 
    n_latent_factors_user,name='User-Embedding')(user_input))
    user_vec = Dropout(dropout_1)(user_vec)

    concat = Concatenate(name='Concat')([movie_vec, user_vec])
    concat = Dropout(dropout_1)(concat)

    x = Dense(hidden_units,name='FullyConnected-1', activation='relu')(concat)
    x = Dropout(dropout_2)(x)
    # x = Dense(50,name='FullyConnected-1', activation='relu')(concat)
    # x = Dropout(0.5)(x)


    ## Se pueden sacar las siguientes dos lineas para no forzar a sigmoidea
    x = Dense(1, activation='sigmoid',name='Activation')(x)
    x = Lambda(lambda z: (max_rating - min_rating) * z + min_rating)(x)
    ##

    checkpointer = ModelCheckpoint(filepath='weights2.hdf5', verbose=1, save_best_only=True, monitor='val_root_mean_squared_error')

    import keras.backend as K 
    def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
    model = Model([user_input, movie_input], x)
    lr = 0.001
    model.compile(Adam(learning_rate=learning_rate), 'mean_squared_error', metrics=[root_mean_squared_error])
    #mlflow.log_param("lr", lr)
    batch_size = 320
    epochs = 5
    #mlflow.log_param("batch_size", batch_size)
    #mlflow.log_param("epochs", epochs)

    history = model.fit([ratings_train.userId, ratings_train.movieId], 
                        ratings_train.rating, 
                        validation_data=([ratings_val.userId, ratings_val.movieId], ratings_val.rating), 
                        batch_size = batch_size,
                        callbacks = [checkpointer],
                        epochs=epochs, verbose=1)
    val_loss, rmse = np.array(model.evaluate([ratings_val.userId, ratings_val.movieId], ratings_val.rating))
    return rmse

In [ ]:
study = optuna.create_study(
    # direction="maximize"
    direction="minimize"
)
study.optimize(objective, n_trials=50)

[I 2024-11-24 20:27:29,289] A new study created in memory with name: no-name-4a116c67-5dd8-483a-9aed-0822cab0fc9f


200948 84432 200948 80204


2024-11-24 20:27:37.131553: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
79996/80001 [============================>.] - ETA: 0s - loss: 1.1458 - root_mean_squared_error: 1.0637
Epoch 1: val_root_mean_squared_error improved from inf to 1.05976, saving model to weights2.hdf5
80001/80001 [==============================] - 346s 4ms/step - loss: 1.1458 - root_mean_squared_error: 1.0637 - val_loss: 1.1321 - val_root_mean_squared_error: 1.0598
Epoch 2/5
79999/80001 [============================>.] - ETA: 0s - loss: 1.1344 - root_mean_squared_error: 1.0614
Epoch 2: val_root_mean_squared_error did not improve from 1.05976
80001/80001 [==============================] - 983s 12ms/step - loss: 1.1344 - root_mean_squared_error: 1.0614 - val_loss: 1.1285 - val_root_mean_squared_error: 1.0602
Epoch 3/5
80000/80001 [============================>.] - ETA: 0s - loss: 1.1306 - root_mean_squared_error: 1.0603
Epoch 3: val_root_mean_squared_error improved from 1.05976 to 1.05928, saving model to weights2.hdf5
80001/80001 [==============================] - 1701s 21ms/s

[I 2024-11-24 21:35:13,472] Trial 0 finished with value: 1.051754117012024 and parameters: {'hidden_units': 171, 'learning_rate': 0.09760134311552048, 'dropout_1': 0.33977068106260316, 'dropout_2': 0.1636598288388696}. Best is trial 0 with value: 1.051754117012024.


...

200948 84432 200948 80258
Epoch 1/5
79996/80001 [============================>.] - ETA: 0s - loss: 1.0154 - root_mean_squared_error: 0.9391
Epoch 1: val_root_mean_squared_error improved from inf to 0.91443, saving model to weights2.hdf5
80001/80001 [==============================] - 349s 4ms/step - loss: 1.0154 - root_mean_squared_error: 0.9391 - val_loss: 0.9685 - val_root_mean_squared_error: 0.9144
Epoch 2/5
80001/80001 [==============================] - ETA: 0s - loss: 0.9926 - root_mean_squared_error: 0.9286
Epoch 2: val_root_mean_squared_error improved from 0.91443 to 0.91077, saving model to weights2.hdf5
80001/80001 [==============================] - 331s 4ms/step - loss: 0.9926 - root_mean_squared_error: 0.9286 - val_loss: 0.9672 - val_root_mean_squared_error: 0.9108
Epoch 3/5
79993/80001 [============================>.] - ETA: 0s - loss: 0.9860 - root_mean_squared_error: 0.9269
Epoch 3: val_root_mean_squared_error did not improve from 0.91077
80001/80001 [=====================

[I 2024-11-25 22:11:52,690] Trial 44 finished with value: 0.898984968662262 and parameters: {'hidden_units': 138, 'learning_rate': 0.015893533005819128, 'dropout_1': 0.10050169140675555, 'dropout_2': 0.1956424453135526}. Best is trial 44 with value: 0.898984968662262.


200948 84432 200948 80307
Epoch 1/5
80000/80001 [============================>.] - ETA: 0s - loss: 1.0045 - root_mean_squared_error: 0.9363
Epoch 1: val_root_mean_squared_error improved from inf to 0.91729, saving model to weights2.hdf5
80001/80001 [==============================] - 342s 4ms/step - loss: 1.0045 - root_mean_squared_error: 0.9363 - val_loss: 0.9660 - val_root_mean_squared_error: 0.9173
Epoch 2/5
79991/80001 [============================>.] - ETA: 0s - loss: 0.9830 - root_mean_squared_error: 0.9256
Epoch 2: val_root_mean_squared_error improved from 0.91729 to 0.91522, saving model to weights2.hdf5
80001/80001 [==============================] - 335s 4ms/step - loss: 0.9830 - root_mean_squared_error: 0.9256 - val_loss: 0.9661 - val_root_mean_squared_error: 0.9152
Epoch 3/5
80001/80001 [==============================] - ETA: 0s - loss: 0.9800 - root_mean_squared_error: 0.9239
Epoch 3: val_root_mean_squared_error improved from 0.91522 to 0.90969, saving model to weights2.hdf5

[I 2024-11-25 22:43:08,256] Trial 45 finished with value: 0.9034157395362854 and parameters: {'hidden_units': 72, 'learning_rate': 0.014640997285745232, 'dropout_1': 0.10067573375831627, 'dropout_2': 0.19493237736381}. Best is trial 44 with value: 0.898984968662262.


200948 84432 200948 80348
Epoch 1/5
80000/80001 [============================>.] - ETA: 0s - loss: 1.1993 - root_mean_squared_error: 0.9875
Epoch 1: val_root_mean_squared_error improved from inf to 0.95767, saving model to weights2.hdf5
80001/80001 [==============================] - 333s 4ms/step - loss: 1.1993 - root_mean_squared_error: 0.9875 - val_loss: 1.1019 - val_root_mean_squared_error: 0.9577
Epoch 2/5
79996/80001 [============================>.] - ETA: 0s - loss: 1.1751 - root_mean_squared_error: 0.9823
Epoch 2: val_root_mean_squared_error did not improve from 0.95767
80001/80001 [==============================] - 353s 4ms/step - loss: 1.1751 - root_mean_squared_error: 0.9823 - val_loss: 1.1541 - val_root_mean_squared_error: 0.9708
Epoch 3/5
11709/80001 [===>..........................] - ETA: 5:53 - loss: 1.1655 - root_mean_squared_error: 0.9842

In [1]:
trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

NameError: name 'study' is not defined

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_contour(study, params=["hidden_units", "learning_rate"])

In [12]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/opt/anaconda3/envs/tp_2/lib/python3.8/site-packages/tensorflow/python/client/session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [13]:
def objective(trial):
    from sklearn.model_selection import train_test_split
    from keras.layers import Input, Embedding, Flatten, Dropout, Concatenate, Dense, Activation, Lambda
    from keras import Model
    from keras.regularizers import l2
    from keras.optimizers import Adam
    
    
    hidden_units = trial.suggest_int("hidden_units", 125, 145)
    learning_rate = 0.015#trial.suggest_float("learning_rate", 1e-2, 1e1, log=True)
    dropout_1 = 0.1#trial.suggest_float("dropout_1", 0.1, 0.4, log=True)
    dropout_2 = 0.2#trial.suggest_float("dropout_2", 0.1, 0.5, log=True)

    ratings_train, ratings_val = train_test_split(ratings, test_size=0.2)
    n_users = int(ratings.userId.nunique())
    n_movies = int(ratings.movieId.nunique())
    n_users_train = int(ratings_train.userId.nunique())
    n_movies_train = int(ratings_train.movieId.nunique())
    print(n_users, n_movies, n_users_train, n_movies_train)
    max_rating = ratings_train['rating'].max()
    min_rating = ratings_train['rating'].min()
    av_rating = ratings_train['rating'].mean()
    max_rating, min_rating, av_rating


    movie_embedding_regularizer = 0.001
    #mlflow.log_param("movie_embedding_regularizer_l2", movie_embedding_regularizer)

    movie_input = Input(shape=[1],name='Item')
    movie_embedding = Embedding(n_movies + 1, n_latent_factors_movie, name='Movie-Embedding', embeddings_regularizer = l2(movie_embedding_regularizer))(movie_input)
    movie_vec = Flatten(name='FlattenMovies')(movie_embedding)
    movie_vec = Dropout(dropout_1)(movie_vec)

    user_input = Input(shape=[1],name='User')
    user_vec = Flatten(name='FlattenUsers')(Embedding(n_users + 1, 
    n_latent_factors_user,name='User-Embedding')(user_input))
    user_vec = Dropout(dropout_1)(user_vec)

    concat = Concatenate(name='Concat')([movie_vec, user_vec])
    concat = Dropout(dropout_1)(concat)

    x = Dense(hidden_units,name='FullyConnected-1', activation='relu')(concat)
    x = Dropout(dropout_2)(x)
    # x = Dense(50,name='FullyConnected-1', activation='relu')(concat)
    # x = Dropout(0.5)(x)


    ## Se pueden sacar las siguientes dos lineas para no forzar a sigmoidea
    x = Dense(1, activation='sigmoid',name='Activation')(x)
    x = Lambda(lambda z: (max_rating - min_rating) * z + min_rating)(x)
    ##

    checkpointer = ModelCheckpoint(filepath='weights2.hdf5', verbose=1, save_best_only=True, monitor='val_root_mean_squared_error')

    import keras.backend as K 
    def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
    model = Model([user_input, movie_input], x)
    lr = 0.001
    model.compile(Adam(learning_rate=learning_rate), 'mean_squared_error', metrics=[root_mean_squared_error])
    #mlflow.log_param("lr", lr)
    batch_size = 320
    epochs = 5
    #mlflow.log_param("batch_size", batch_size)
    #mlflow.log_param("epochs", epochs)

    history = model.fit([ratings_train.userId, ratings_train.movieId], 
                        ratings_train.rating, 
                        validation_data=([ratings_val.userId, ratings_val.movieId], ratings_val.rating), 
                        batch_size = batch_size,
                        callbacks = [checkpointer],
                        epochs=epochs, verbose=1)
    val_loss, rmse = np.array(model.evaluate([ratings_val.userId, ratings_val.movieId], ratings_val.rating))
    return rmse

In [14]:
study = optuna.create_study(
    # direction="maximize"
    direction="minimize"
)
study.optimize(objective, n_trials=20)

[I 2024-11-25 23:13:02,281] A new study created in memory with name: no-name-2241cff9-8df2-43ef-96aa-f84fe224fcb4


200948 84432 200948 80245
Epoch 1/5
79999/80001 [============================>.] - ETA: 0s - loss: 1.0109 - root_mean_squared_error: 0.9382
Epoch 1: val_root_mean_squared_error improved from inf to 0.91946, saving model to weights2.hdf5
80001/80001 [==============================] - 434s 5ms/step - loss: 1.0109 - root_mean_squared_error: 0.9382 - val_loss: 0.9770 - val_root_mean_squared_error: 0.9195
Epoch 2/5
80001/80001 [==============================] - ETA: 0s - loss: 0.9882 - root_mean_squared_error: 0.9272
Epoch 2: val_root_mean_squared_error improved from 0.91946 to 0.90972, saving model to weights2.hdf5
80001/80001 [==============================] - 613s 8ms/step - loss: 0.9882 - root_mean_squared_error: 0.9272 - val_loss: 0.9569 - val_root_mean_squared_error: 0.9097
Epoch 3/5
80001/80001 [==============================] - ETA: 0s - loss: 0.9845 - root_mean_squared_error: 0.9253
Epoch 3: val_root_mean_squared_error did not improve from 0.90972
80001/80001 [=====================

[I 2024-11-26 00:07:42,855] Trial 0 finished with value: 0.8949152827262878 and parameters: {'hidden_units': 126}. Best is trial 0 with value: 0.8949152827262878.


200948 84432 200948 80242
Epoch 1/5
79995/80001 [============================>.] - ETA: 0s - loss: 1.0063 - root_mean_squared_error: 0.9371
Epoch 1: val_root_mean_squared_error improved from inf to 0.91388, saving model to weights2.hdf5
80001/80001 [==============================] - 449s 6ms/step - loss: 1.0063 - root_mean_squared_error: 0.9371 - val_loss: 0.9585 - val_root_mean_squared_error: 0.9139
Epoch 2/5
80001/80001 [==============================] - ETA: 0s - loss: 0.9840 - root_mean_squared_error: 0.9262
Epoch 2: val_root_mean_squared_error improved from 0.91388 to 0.91373, saving model to weights2.hdf5
80001/80001 [==============================] - 508s 6ms/step - loss: 0.9840 - root_mean_squared_error: 0.9262 - val_loss: 0.9725 - val_root_mean_squared_error: 0.9137
Epoch 3/5
80000/80001 [============================>.] - ETA: 0s - loss: 0.9818 - root_mean_squared_error: 0.9243
Epoch 3: val_root_mean_squared_error improved from 0.91373 to 0.91023, saving model to weights2.hdf5

[I 2024-11-26 00:59:53,486] Trial 1 finished with value: 0.9015427231788635 and parameters: {'hidden_units': 132}. Best is trial 0 with value: 0.8949152827262878.


...

200948 84432 200948 80281
Epoch 1/5
80000/80001 [============================>.] - ETA: 0s - loss: 1.0072 - root_mean_squared_error: 0.9372
Epoch 1: val_root_mean_squared_error improved from inf to 0.91393, saving model to weights2.hdf5
80001/80001 [==============================] - 325s 4ms/step - loss: 1.0072 - root_mean_squared_error: 0.9372 - val_loss: 0.9711 - val_root_mean_squared_error: 0.9139
Epoch 2/5
79998/80001 [============================>.] - ETA: 0s - loss: 0.9861 - root_mean_squared_error: 0.9262
Epoch 2: val_root_mean_squared_error improved from 0.91393 to 0.90982, saving model to weights2.hdf5
80001/80001 [==============================] - 323s 4ms/step - loss: 0.9861 - root_mean_squared_error: 0.9262 - val_loss: 0.9636 - val_root_mean_squared_error: 0.9098
Epoch 3/5
79990/80001 [============================>.] - ETA: 0s - loss: 0.9820 - root_mean_squared_error: 0.9243
Epoch 3: val_root_mean_squared_error did not improve from 0.90982
80001/80001 [=====================

[I 2024-11-26 06:52:51,327] Trial 12 finished with value: 0.9029991030693054 and parameters: {'hidden_units': 128}. Best is trial 0 with value: 0.8949152827262878.


200948 84432 200948 80196
Epoch 1/5
15863/80001 [====>.........................] - ETA: 4:39 - loss: 1.0482 - root_mean_squared_error: 0.9553

[W 2024-11-26 06:54:14,622] Trial 13 failed with parameters: {'hidden_units': 129} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tp_2/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/7f/19f36bv57_72qpq3mfj1x6240000gn/T/ipykernel_18302/2477498198.py", line 68, in objective
    history = model.fit([ratings_train.userId, ratings_train.movieId],
  File "/opt/anaconda3/envs/tp_2/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/opt/anaconda3/envs/tp_2/lib/python3.8/site-packages/keras/engine/training.py", line 1650, in fit
    tmp_logs = self.train_function(iterator)
  File "/opt/anaconda3/envs/tp_2/lib/python3.8/site-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/opt/anaconda3/envs/tp_2/li

KeyboardInterrupt: 

In [16]:
trial = study.best_trial

print("RMSE: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

RMSE: 0.8949152827262878
Best hyperparameters: {'hidden_units': 126}
